### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP на implicit данных

Мягкий дедлайн 13 Октября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 20 Октября (Итоговая проверка)

In [1]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens

In [2]:
from utils.factorization import SVD, ALS, BPR

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [3]:
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [4]:
movie_info = pd.read_csv('ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [5]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [6]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [7]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [8]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [9]:
# model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [10]:
# model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [11]:
# movie_info.head(5)

In [12]:
# get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
#                                         for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [13]:
# get_similars(1, model)

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [14]:
# get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
#                                             for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [15]:
# get_user_history(4, implicit_ratings)

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [16]:
# get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
#                                                for x in model.recommend(user_id, user_item_csr)]

In [17]:
# get_recommendations(4, model)

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

In [18]:
def test_model(model, test_id=1, user_id=4, n=10):
    target = movie_info[movie_info.movie_id == test_id]
    sims_indxs = model.get_similar_items_indxs(test_id, n)

    print(f'For {target.name.item()} ({target.category.item()}) found these {n} similiars):')
    for indx in sims_indxs:
            similar_item = movie_info[movie_info.movie_id == indx]
            if len(similar_item) == 0:
                print("movie doesn't exist")
            else:
                print(f'\t {similar_item.name.item()} ({similar_item.category.item()})')
            
    print('____________________________________________')

    recs = model.recommend(user_id, n)
    print(f"For user {user_id} found these movies:")
    for idx in recs:
            rec = movie_info[movie_info.movie_id == idx]
            if len(rec) == 0:
                print("movie doesn't exist")
            else:
                print(f'\t {rec.name.item()} ({rec.category.item()})')

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [19]:
user_item_exp = sp.coo_matrix((ratings["rating"], (ratings["user_id"], ratings["movie_id"])))

In [20]:
model_svd = SVD(64, max_iters=500, lr=1e-4, verbose=100, lam=1e-3)
model_svd.fit(user_item_exp);

Iter: 100, score: 0.9353352486953457
Iter: 200, score: 0.9071451179754194
Iter: 300, score: 0.8916444433343037
Iter: 400, score: 0.8653709773786614
Iter: 500, score: 0.834296803654223


In [21]:
test_model(model_svd)

For Toy Story (1995) (Animation|Children's|Comedy) found these 10 similiars):
	 Toy Story (1995) (Animation|Children's|Comedy)
	 Toy Story 2 (1999) (Animation|Children's|Comedy)
	 Bug's Life, A (1998) (Animation|Children's|Comedy)
	 Beauty and the Beast (1991) (Animation|Children's|Musical)
	 Aladdin (1992) (Animation|Children's|Comedy|Musical)
	 Little Mermaid, The (1989) (Animation|Children's|Comedy|Musical|Romance)
	 Lion King, The (1994) (Animation|Children's|Musical)
	 Untouchables, The (1987) (Action|Crime|Drama)
	 Father of the Bride (1950) (Comedy)
	 Charlotte's Web (1973) (Animation|Children's)
____________________________________________
For user 4 found these movies:
	 Casablanca (1942) (Drama|Romance|War)
	 Close Shave, A (1995) (Animation|Comedy|Thriller)
	 Paths of Glory (1957) (Drama|War)
	 Sunset Blvd. (a.k.a. Sunset Boulevard) (1950) (Film-Noir)
	 Usual Suspects, The (1995) (Crime|Thriller)
	 Wrong Trousers, The (1993) (Animation|Comedy)
	 Dr. Strangelove or: How I Lea

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [22]:
model_als = ALS(64, max_iters=15, lr=1e-5, verbose=2, lam=1e-5)
model_als.fit(user_item_csr)

test_model(model_als)

Iter: 2, score: 0.655510277563349
Iter: 4, score: 0.6417459483121501
Iter: 6, score: 0.6395552778758652
Iter: 8, score: 0.6388635030771348
Iter: 10, score: 0.6385687225089174
Iter: 12, score: 0.638422092014349
Iter: 14, score: 0.638342659649418
For Toy Story (1995) (Animation|Children's|Comedy) found these 10 similiars):
	 Toy Story (1995) (Animation|Children's|Comedy)
	 Toy Story 2 (1999) (Animation|Children's|Comedy)
	 Bug's Life, A (1998) (Animation|Children's|Comedy)
	 Aladdin (1992) (Animation|Children's|Comedy|Musical)
	 Babe (1995) (Children's|Comedy|Drama)
	 Lion King, The (1994) (Animation|Children's|Musical)
	 Hercules (1997) (Adventure|Animation|Children's|Comedy|Musical)
	 Tarzan (1999) (Animation|Children's)
	 Pleasantville (1998) (Comedy)
	 Beauty and the Beast (1991) (Animation|Children's|Musical)
____________________________________________
For user 4 found these movies:
	 Goldfinger (1964) (Action)
	 Indiana Jones and the Last Crusade (1989) (Action|Adventure)
	 Die Ha

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [23]:
model_bpr = BPR(64, max_iters=int(5e3), lr=1e-1, verbose=1000, lam=1e-2)
model_bpr.fit(user_item_csr, batch_size=2000)

test_model(model_bpr)


Iter: 1000, score: 0.9119731469755307
Iter: 2000, score: 0.8791850244806902
Iter: 3000, score: 0.9076135471795086
Iter: 4000, score: 0.9133044352288551
Iter: 5000, score: 0.9007502908715451
For Toy Story (1995) (Animation|Children's|Comedy) found these 10 similiars):
	 Toy Story (1995) (Animation|Children's|Comedy)
	 Toy Story 2 (1999) (Animation|Children's|Comedy)
	 Harriet the Spy (1996) (Children's|Comedy)
	 Babe (1995) (Children's|Comedy|Drama)
	 Thumbelina (1994) (Animation|Children's)
	 Jungle2Jungle (a.k.a. Jungle 2 Jungle) (1997) (Children's|Comedy)
	 Hercules (1997) (Adventure|Animation|Children's|Comedy|Musical)
	 Little Big League (1994) (Children's|Comedy)
	 Mighty Morphin Power Rangers: The Movie (1995) (Action|Children's)
	 Doug's 1st Movie (1999) (Animation|Children's)
____________________________________________
For user 4 found these movies:
	 Star Wars: Episode V - The Empire Strikes Back (1980) (Action|Adventure|Drama|Sci-Fi|War)
	 Good, The Bad and The Ugly, The (19

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных